# Label side

In [2]:
import numpy as np
from jesse import helpers, research

_, candles = research.get_candles(
    "Binance Perpetual Futures",
    "BTC-USDT",
    "1m",
    helpers.date_to_timestamp("2022-07-01"),
    helpers.date_to_timestamp("2025-07-25"),
    warmup_candles_num=0,
    caching=False,
    is_for_jesse=False,
)

candles.shape

(1612800, 6)

In [3]:
np.save("data/btc_1m.npy", candles)
del candles

In [1]:
import numpy as np

candles = np.load("data/btc_1m.npy")
candles.shape

(1657440, 6)

In [2]:
from src.bars.fusion.deap_v1 import DeapBarV1

bar_container = DeapBarV1(max_bars=500000)
bar_container.update_with_candles(candles)
merged_bar = bar_container.get_fusion_bars()
merged_bar.shape

(5789, 6)

In [3]:
np.save("data/bar_deap_v1.npy", merged_bar)

In [4]:
from research.labeler.gmm_labeler import GMMLabeler

labeler_L4 = GMMLabeler(merged_bar, 4)
labeler_L4.plot_label_on_candles()

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/jesse/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[I 2025-09-07 15:37:13,136] A new study created in memory with name: no-name-7e0f3960-0084-4d92-a4cc-368a42bdf648
[I 2025-09-07 15:37:13,715] Trial 0 finished with value: 24.328785532080833 and parameters: {'random_state': 71}. Best is trial 0 with value: 24.328785532080833.
[I 2025-09-07 15:37:14,321] Trial 1 finished with value: 24.328785532080833 and parameters: {'random_state': 331}. Best is trial 0 with value: 24.328785532080833.
[I 2025-09-07 15:37:14,852] Trial 2 finished with value: 1.7704358515294387 and parameters: {'random_state': 748}. Best is trial 0 with value: 24.328785532080833.
[I 2025-09-07 15:37:15,611] Trial 

In [11]:
# labeler_L4.plot_label_returns()

In [5]:
labeler_L5 = GMMLabeler(merged_bar, 5)
labeler_L5.plot_label_on_candles()

[I 2025-09-07 15:37:25,126] A new study created in memory with name: no-name-e4a0aa5e-991c-4c25-bc29-a0153ec025c1
[I 2025-09-07 15:37:25,755] Trial 0 finished with value: 21.672070756241887 and parameters: {'random_state': 225}. Best is trial 0 with value: 21.672070756241887.
[I 2025-09-07 15:37:26,417] Trial 1 finished with value: 1.7623536344764892 and parameters: {'random_state': 97}. Best is trial 0 with value: 21.672070756241887.
[I 2025-09-07 15:37:27,022] Trial 2 finished with value: 21.672070756241887 and parameters: {'random_state': 488}. Best is trial 0 with value: 21.672070756241887.
[I 2025-09-07 15:37:27,562] Trial 3 finished with value: 1.7623536344764887 and parameters: {'random_state': 877}. Best is trial 0 with value: 21.672070756241887.
[I 2025-09-07 15:37:28,232] Trial 4 finished with value: 21.672070756241887 and parameters: {'random_state': 585}. Best is trial 0 with value: 21.672070756241887.
[I 2025-09-07 15:37:28,970] Trial 5 finished with value: 21.672070756241

In [6]:
labeler_L6 = GMMLabeler(merged_bar, 6)
labeler_L6.plot_label_on_candles()

[I 2025-09-07 15:37:34,800] A new study created in memory with name: no-name-7d8b58e3-ccbb-4011-8fe0-0eb0cc693353
[I 2025-09-07 15:37:35,749] Trial 0 finished with value: 21.783773068854025 and parameters: {'random_state': 314}. Best is trial 0 with value: 21.783773068854025.
[I 2025-09-07 15:37:36,578] Trial 1 finished with value: 1.7657961912320714 and parameters: {'random_state': 326}. Best is trial 0 with value: 21.783773068854025.
[I 2025-09-07 15:37:37,314] Trial 2 finished with value: 1.7657961912320714 and parameters: {'random_state': 244}. Best is trial 0 with value: 21.783773068854025.
[I 2025-09-07 15:37:38,253] Trial 3 finished with value: 1.7657961912320712 and parameters: {'random_state': 563}. Best is trial 0 with value: 21.783773068854025.
[I 2025-09-07 15:37:38,925] Trial 4 finished with value: 20.742782785882163 and parameters: {'random_state': 844}. Best is trial 0 with value: 21.783773068854025.
[I 2025-09-07 15:37:39,540] Trial 5 finished with value: 17.70276757452

In [7]:
for labeler, idx in zip([labeler_L4, labeler_L5, labeler_L6], [4, 5, 6]):
    label_hard = labeler.label_hard_state
    print(f"{label_hard.shape = }")
    np.save(f"data/label_hard_L{idx}.npy", label_hard)
    label_direction_force = labeler.label_direction_force
    print(f"{label_direction_force.shape = }")
    np.save(f"data/label_direction_L{idx}.npy", label_direction_force)

label_hard.shape = (5785,)
label_direction_force.shape = (5785,)
label_hard.shape = (5784,)
label_direction_force.shape = (5784,)
label_hard.shape = (5783,)
label_direction_force.shape = (5783,)


# Label meta

In [1]:
import numpy as np
import pandas as pd
from strategies.BinanceBtcEntropyBarV1.config import SIDE
from strategies.BinanceBtcEntropyBarV1.config import get_side_model

side_model = get_side_model(False)

merged_bar = np.load("data/merged_bar.npy")

df_features = pd.read_parquet("data/features.parquet")

side_res = side_model.predict(df_features[SIDE])

side_pred_label = np.where(side_res > 0.5, 1, -1)
print(side_pred_label.shape)

print(np.unique(side_pred_label, return_counts=True))

(13219,)
(array([-1,  1]), array([5080, 8139]))


In [2]:
close_prices = merged_bar[:, 2]
len_gap = len(close_prices) - len(side_pred_label)
close_prices = close_prices[len_gap - 1 : -1]

assert len(close_prices) == len(side_pred_label)

In [3]:
meta_label = np.zeros(len(side_pred_label))

TRADE_FEE = 0.05 / 100

start_idx = 0
cumsum_ret = 0
start_price = 0
for idx, (i, p) in enumerate(zip(side_pred_label, close_prices)):
    if i == 1 or i == -1:
        if idx == 0:
            # 开始持仓
            start_idx = idx
            start_price = p
            cumsum_ret -= TRADE_FEE
        elif side_pred_label[idx - 1] != i:
            # 反向持仓，先结算收益
            cumsum_ret -= TRADE_FEE
            cumsum_ret += np.log(p / start_price) * side_pred_label[idx - 1]
            if cumsum_ret > 0:
                meta_label[start_idx:idx] = 1
            cumsum_ret = 0
            start_price = p
            start_idx = idx
            cumsum_ret -= TRADE_FEE
        else:
            # 继续持仓
            continue
    else:
        raise ValueError(f"side_pred_label[{idx}] = {i} is not valid")
else:
    last_price = merged_bar[-1, 2]
    # 结算最后一根bar的持仓, 可能还没有结算，所以先不加trade fee
    if i == side_pred_label[idx - 1]:
        # 已经开仓，结算
        cumsum_ret += np.log(last_price / start_price) * side_pred_label[idx - 1]
        if cumsum_ret > 0:
            meta_label[start_idx:] = 1
    else:
        # 反向开仓
        cumsum_ret -= TRADE_FEE
        cumsum_ret += np.log(last_price / start_price) * side_pred_label[idx - 1]
        if cumsum_ret > 0:
            meta_label[start_idx:] = 1


print(np.unique(meta_label, return_counts=True))

(array([0., 1.]), array([  669, 12550]))


In [4]:
np.save("data/label_meta.npy", meta_label)